**2025W 301599-1 Praktikum Bioinformatik**  
Julianne Weiß  
Sarah Strolz  
Sebastian Rossböck
# Comparison of Sequence-Based and Structure-Based RNA Alignments
Sequence-based and structure-aware alignment strategies are compared, and the effect of manual inspection and structure-guided correction on alignment
quality is evaluated.

In [55]:
import sys
import os
#Helper functions for reading FASTA / STOCKHOLM and CLUSTAL
#and for calculating Sum of Pairs Score
def read_fasta_alignment(filename):
    sequences = {}
    current_id = None
    
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                current_id = line[1:].split()[0]
                sequences[current_id] = ""
            elif current_id is not None:
                sequences[current_id] += line
    
    return sequences


def read_clustal_alignment(filename):
    sequences = {}
    
    with open(filename, "r") as f:
        for line in f:
            line = line.rstrip()
            # Skip header, empty lines, and conservation lines
            if not line or line.startswith("CLUSTAL") or line.startswith(" ") or "*" in line or ":" in line:
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                seq_id = parts[0]
                seq_chunk = parts[1]
                
                if seq_id not in sequences:
                    sequences[seq_id] = ""
                sequences[seq_id] += seq_chunk
    
    return sequences


def read_stockholm_alignment(filename):
    sequences = {}
    
    with open(filename, "r") as f:
        for line in f:
            line = line.rstrip()
            # Skip header lines and annotation lines
            if line.startswith("#") or line.startswith("//") or not line:
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                seq_id = parts[0]
                seq = parts[1]
                
                if seq_id not in sequences:
                    sequences[seq_id] = ""
                sequences[seq_id] += seq
    
    return sequences


def calculate_sps(reference_aln, test_aln):
    """
    Calculate Sum-of-Pairs Score (SPS)
    
    SPS = (number of correctly aligned residue pairs) / (total pairs in reference)
    """
    # Find common sequence IDs
    common_ids = set(reference_aln.keys()) & set(test_aln.keys())
    
    if len(common_ids) < 2:
        print(f"Warning: Only {len(common_ids)} common sequences found")
        return 0.0
    
    common_ids = sorted(common_ids)
    
    # Check alignment lengths
    ref_len = len(next(iter(reference_aln.values())))
    test_len = len(next(iter(test_aln.values())))
    
    print(f"Reference alignment length: {ref_len}")
    print(f"Test alignment length: {test_len}")
    print(f"Common sequences: {len(common_ids)}")
    
    # Count correctly aligned pairs
    total_pairs = 0
    correct_pairs = 0
    
    # For each column position in reference
    for ref_col in range(ref_len):
        # For each pair of sequences
        for i in range(len(common_ids)):
            for j in range(i + 1, len(common_ids)):
                seq1_id = common_ids[i]
                seq2_id = common_ids[j]
                
                # Get residues at this column in reference
                ref_res1 = reference_aln[seq1_id][ref_col]
                ref_res2 = reference_aln[seq2_id][ref_col]
                
                # Skip if either is a gap in reference
                if ref_res1 == '-' or ref_res2 == '-':
                    continue
                
                total_pairs += 1
                
                # Find these residues in test alignment
                # We need to find the corresponding non-gap positions
                test_pos1 = find_residue_position(test_aln[seq1_id], reference_aln[seq1_id], ref_col)
                test_pos2 = find_residue_position(test_aln[seq2_id], reference_aln[seq2_id], ref_col)
                
                # Check if they're aligned in test (same column position)
                if test_pos1 is not None and test_pos2 is not None and test_pos1 == test_pos2:
                    correct_pairs += 1
    
    if total_pairs == 0:
        return 0.0
    
    sps = correct_pairs / total_pairs
    print(f"Correct pairs: {correct_pairs}")
    print(f"Total pairs: {total_pairs}")
    
    return sps


def find_residue_position(test_seq, ref_seq, ref_col):
    """
    Find where the residue at ref_col in ref_seq appears in test_seq
    Returns the column position in test_seq, or None if not found
    """
    # Count non-gap characters up to ref_col in reference
    ref_residue_count = 0
    for i in range(ref_col + 1):
        if ref_seq[i] != '-':
            ref_residue_count += 1
    
    # Find the same residue number in test sequence
    test_residue_count = 0
    for test_col in range(len(test_seq)):
        if test_seq[test_col] != '-':
            test_residue_count += 1
            if test_residue_count == ref_residue_count:
                return test_col
    
    return None

def get_sci(filename):
    """Extract Structure Conservation Index from RNAalifold output"""
    with open(filename, "r") as f:
        for line in f:
            if "[sci =" in line:
                # Format: (...) [sci = 0.8020]
                sci_str = line.split("[sci =")[1].split("]")[0].strip()
                return float(sci_str)
    return None

# Extract bit scores from cmsearch output
def extract_bit_scores(filename):
    """Extract bit scores from cmsearch output"""
    bit_scores = []
    
    with open(filename, "r") as f:
        for line in f:
            line_stripped = line.strip()
            
            # Look for lines that start with (number) followed by ! or ?
            if line_stripped.startswith("("):
                parts = line_stripped.split()
                # Format: (rank) ! E-value score ...
                if len(parts) >= 4 and (parts[1] == "!" or parts[1] == "?"):
                    try:
                        # Fourth element is the bit score
                        bit_score = float(parts[3])
                        bit_scores.append(bit_score)
                    except:
                        pass
    
    return bit_scores

In [44]:
#Download seed alignment and all sequences for family
print("-------------------------------------")
print("------------Download Data------------")
print("-------------------------------------")
# Download seed alignment in fasta
!wget https://rfam.org/family/RF01988/alignment/fasta -O RF01988_seed.fa
# Download seed alignment without gap (ungapped) (these will be needed by clustalw and locarna)
!wget https://rfam.org/family/RF01988/alignment/fastau -O RF01988_seed_ungapped.fa
# Download and extract all sequences
!wget https://ftp.ebi.ac.uk/pub/databases/Rfam/15.1/fasta_files/RF01988.fa.gz
!wsl gunzip RF01988.fa.gz

-------------------------------------
------------Download Data------------
-------------------------------------


--2026-02-08 16:45:26--  https://rfam.org/family/RF01988/alignment/fasta
Resolving rfam.org (rfam.org)... 193.62.193.83
Connecting to rfam.org (rfam.org)|193.62.193.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330 [text/plain]
Saving to: 'RF01988_seed.fa'

     0K                                                       100% 95,4M=0s

2026-02-08 16:45:27 (95,4 MB/s) - 'RF01988_seed.fa' saved [330/330]

--2026-02-08 16:45:27--  https://rfam.org/family/RF01988/alignment/fastau
Resolving rfam.org (rfam.org)... 193.62.193.83
Connecting to rfam.org (rfam.org)|193.62.193.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330 [text/plain]
Saving to: 'RF01988_seed_ungapped.fa'

     0K                                                       100%  210M=0s

2026-02-08 16:45:29 (210 MB/s) - 'RF01988_seed_ungapped.fa' saved [330/330]

--2026-02-08 16:45:29--  https://rfam.org/family/RF01988/alignment?format=stockholm
Resolving rfam.org (rfam.org

In [49]:
#create the full alignment
# Download the Rfam covariance model
!wsl wget https://rfam.org/family/RF01988/cm -O RF01988_rfam.cm
# Align all sequences to the covariance model to create the full alignment
!wsl cmalign RF01988_rfam.cm RF01988.fa > RF01988_full_alignment.sto

--2026-02-08 16:49:18--  https://rfam.org/family/RF01988/cm
Resolving rfam.org (rfam.org)... 193.62.193.83
Connecting to rfam.org (rfam.org)|193.62.193.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37874 (37K) [text/plain]
Saving to: â€˜RF01988_rfam.cmâ€™

     0K .......... .......... .......... ......               100%  920K=0.04s

2026-02-08 16:49:18 (920 KB/s) - â€˜RF01988_rfam.cmâ€™ saved [37874/37874]



In [9]:
#Create Sequence-based alignments are produced with Clustal W
#Followed by consensus structure prediction using RNAalifold
print("---------------------------------------------------")
print("----------Create Sequence Based Alignment----------")
print("---------------------------------------------------")
#Read the Sequences from the ungapped seed file and save it to a file, to prepare it as input for clustalw
sequences = {}
current_id = None

#read sequenceId / sequence from file
with open("RF01988_seed_ungapped.fa","r") as f:
    for line in f:
        #print(line)
        if line.startswith(">"):
            current_id = line[1:].split()[0]
            sequences[current_id] = ""
        elif current_id is not None:
            sequences[current_id] += line

#print it to make sure we have data (we expect 4-20 sequences)
for seq_id, seq in sequences.items():
    print(f"{seq_id}: {len(seq)} nt")

#write sequences to file
with open("RF01988_for_clustalw.fa", "w") as f:
    for seq_id, seq in sequences.items():
        f.write(f">{seq_id}\n{seq}\n")
        
#Run clustalw
!wsl clustalw -infile=RF01988_for_clustalw.fa -outfile=RF01988_clustalw.aln -output=CLUSTAL

#Use the clustalw alignment as input for RNAalifold to predict consensus structure
# --sci flag for structure conservation index
!wsl RNAalifold --sci RF01988_clustalw.aln > RF01988_clustalw_alifold.out

#show RNAalifold output
print("-------------------------------------")
print("----------RNAalifold output----------")
print("-------------------------------------")
with open("RF01988_clustalw_alifold.out","r") as f:
    for line in f:
        print(line)

U00096.2/4296977-4296923: 56 nt
CU928158.2/4397661-4397715: 56 nt
ABXW01000042.1/36779-36725: 56 nt
BAAW01013798.1/777-831: 56 nt



 CLUSTAL 2.1 Multiple Sequence Alignments


Sequence format is Pearson
Sequence 1: U00096.2/4296977-4296923      55 bp
Sequence 2: CU928158.2/4397661-4397715    55 bp
Sequence 3: ABXW01000042.1/36779-36725    55 bp
Sequence 4: BAAW01013798.1/777-831        55 bp
Start of Pairwise alignments
Aligning...

Sequences (1:2) Aligned. Score:  94
Sequences (1:3) Aligned. Score:  92
Sequences (1:4) Aligned. Score:  87
Sequences (2:3) Aligned. Score:  89
Sequences (2:4) Aligned. Score:  89
Sequences (3:4) Aligned. Score:  81
Guide tree file created:   [RF01988_for_clustalw.dnd]

There are 3 groups
Start of Multiple Alignment

Aligning...
Group 1: Sequences:   2      Score:959
Group 2: Sequences:   2      Score:978
Group 3: Sequences:   4      Score:916
Alignment Score 1922

CLUSTAL-Alignment file created  [RF01988_clustalw.aln]

------------------------------------

4 sequences; length of alignment 55.


In [10]:
#what are we doing here?
print("----------------------------------------------------")
print("----------Create Structure Based Alignment----------")
print("----------------------------------------------------")

#we use locarna to create the structure based alignment
# --consensus-structture alifold means, that it used RNAalifold internally for the consensus structure. This way we can compare it better with the clustalw/RNAalifold approach
# --stockholm means that the output is in the stockholm format, which we will need for infernal
!wsl mlocarna --stockholm --consensus-structure alifold RF01988_seed_ungapped.fa

----------------------------------------------------
----------Create Structure Based Alignment----------
----------------------------------------------------
mLocARNA --- multiple Local (and global) Alignment of RNA --- LocARNA 2.0.1


U00096.2/4296977-4296923   GUGUCUGACACGGCCCAUCGGUUGCAGGUCUGCACCAAUCGGUCGGUAAUGGCGC
CU928158.2/4397661-4397715 GCGUCUGACACGGCCCAUCGGUUGCAGGUCUGCACCAGUCGGUCGGUAACGGCGC
ABXW01000042.1/36779-36725 GUGUCUGACACGGCCCUUCGGUUGCAGGUCUGCAGCGUUCGGUCGGUAAUGGCGC
BAAW01013798.1/777-831     GCGUCUGACACGGGCCAUCGGUUGCCGGUCUGCGCCCAUCGGUCGGUGACGGCGC

alifold                    (((((..((.(((((.((.(((.(((....)))))).)).)))))))...))))) (-27.19 = -17.30 +  -9.89)
Results written to target directory /mnt/c/Users/seppo/Desktop/ws25_bioinf/RF01988_seed_ungapped.out.


In [37]:
#Now we can compare the two
#We load the reference alignment and create the SPS of the clustalw alignment and the locarna alignment with it.
#to see how well the alignments compare to the reference alignment from rfam

# Load reference alignment (Rfam seed)
print("Loading reference alignment...")
reference = read_fasta_alignment("RF01988_seed.fa")

# Load Clustal W alignment
print("\n--- Clustal W Alignment ---")
clustalw = read_clustal_alignment("RF01988_clustalw.aln")
sps_clustalw = calculate_sps(reference, clustalw)
print(f"SPS for Clustal W: {sps_clustalw:.4f}")

# Load LocARNA alignment
print("\n--- LocARNA Alignment ---")
locarna = read_stockholm_alignment("RF01988_seed_ungapped.out/results/result.stk")
sps_locarna = calculate_sps(reference, locarna)
print(f"SPS for LocARNA: {sps_locarna:.4f}")

# Load SCI values
sci_clustalw = get_sci("RF01988_clustalw_alifold.out")

#We need to run RNAalifold on the locarna alignment to get theh SCI (can we do this better?)
!wsl RNAalifold --sci RF01988_seed_ungapped.out/results/result.stk > RF01988_locarna_alifold.out

sci_locarna = get_sci("RF01988_locarna_alifold.out")

# Summary
print("RESULTS")
print(f"Clustal W: SPS = {sps_clustalw:.4f}, SCI = {sci_clustalw:.4f}")
print(f"LocARNA:   SPS = {sps_locarna:.4f}, SCI = {sci_locarna:.4f}")

Loading reference alignment...

--- Clustal W Alignment ---
Reference alignment length: 55
Test alignment length: 55
Common sequences: 4
Correct pairs: 330
Total pairs: 330
SPS for Clustal W: 1.0000

--- LocARNA Alignment ---
Reference alignment length: 55
Test alignment length: 55
Common sequences: 4
Correct pairs: 330
Total pairs: 330
SPS for LocARNA: 1.0000

=== PHASE 4 RESULTS ===
Clustal W: SPS = 1.0000, SCI = 0.8020
LocARNA:   SPS = 1.0000, SCI = 0.8020


4 sequences; length of alignment 55.


In [39]:
#Building the covariance models
print("----------------------------------------------------")
print("---------------Build Covariance Models--------------")
print("----------------------------------------------------")
#Infernal requires Stockholm format with a consensus structure annotation
#locarna already provides this format, but for clustalw, we need to create it.

# Clustal W alignment with consensus structure
def create_stockholm_with_structure(alignment_dict, structure, output_file):
    with open(output_file, "w") as f:
        f.write("# STOCKHOLM 1.0\n")
        f.write(f"#=GF SQ {len(alignment_dict)}\n")
        
        # Write sequences
        for seq_id, seq in alignment_dict.items():
            f.write(f"{seq_id:<30} {seq}\n")
        
        # Write consensus structure
        f.write(f"#=GC SS_cons{' '*19} {structure}\n")
        f.write("//\n")

# Create Stockholm file for Clustal W alignment
#the consensus structure was directly copied from the RNAalifold output above
consensus_structure = "(((((..((.(((((....(((.(((....))))))....)))))))...)))))"
create_stockholm_with_structure(clustalw, consensus_structure, "RF01988_clustalw.stk")

# LocARNA already has Stockholm format with structure, so just copy it
!wsl cp RF01988_seed_ungapped.out/results/result.stk RF01988_locarna.stk

Created RF01988_clustalw.stk
Created RF01988_locarna.stk


In [40]:
# Build CM from Clustal W alignment
!wsl cmbuild RF01988_clustalw.cm RF01988_clustalw.stk

# Build CM from LocARNA alignment
!wsl cmbuild RF01988_locarna.cm RF01988_locarna.stk

# cmbuild :: covariance model construction from multiple sequence alignments
# INFERNAL 1.1.5 (Sep 2023)
# Copyright (C) 2023 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# CM file:                                            RF01988_clustalw.cm
# alignment file:                                     RF01988_clustalw.stk
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#                                                                      rel entropy
#                                                                      -----------
# idx    name                     nseq eff_nseq   alen  clen  bps bifs    CM   HMM description
# ------ -------------------- -------- -------- ------ ----- ---- ---- ----- ----- -----------
       1 RF01988_clustalw            4     2.19     55    55   18    0 1.010 0.801 
#
# CPU time: 0.07u 0.00s 00:00:00.07 Elapsed: 0

In [50]:
#Prepare test sequences:
# Load the full alignment, ie all known sequences belonging to this family
full_aln = read_stockholm_alignment("RF01988_full_alignment.sto")

print(f"Full alignment: {len(full_aln)} sequences")
print(f"Seed alignment: {len(reference)} sequences")

#we only want the sequences that were not used in the seed. -> they should be independent
seed_ids = set(reference.keys())
full_ids = set(full_aln.keys())
independent_ids = full_ids - seed_ids

print(f"independent_ids: {len(independent_ids)}")

# Write independent sequences to fasta file
with open("RF01988_independent.fa", "w") as f:
    for seq_id in independent_ids:
        seq = full_aln[seq_id].replace("-", "")  # Remove gaps
        f.write(f">{seq_id}\n{seq}\n")

print("Created RF01988_independent.fa")

Full alignment: 70 sequences
Seed alignment: 4 sequences
independent_ids: 66
Created RF01988_independent.fa


In [51]:
#run cmcalibrate, a cm file must be calibrated before it can be used with cmsearch
!wsl cmcalibrate RF01988_clustalw.cm
#!wsl cmcalibrate RF01988_locarna.cm

# cmcalibrate :: fit exponential tails for CM E-values
# INFERNAL 1.1.5 (Sep 2023)
# Copyright (C) 2023 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# CM file:                                     RF01988_clustalw.cm
# number of worker threads:                    4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#
# Calibrating CM(s):
#
#                        predicted                                                   actual   
#                       running time              percent complete                running time
# model name            (hr:min:sec)  [........25........50........75..........]  (hr:min:sec)
# --------------------  ------------  ------------------------------------------  ------------
  RF01988_clustalw          00:02:25  [========================================]      00:04:43
#
# Calibration summary statistics:
#
#    

In [58]:
#run infernals cmsearch (it searches sequences and tries to find matches to the covariance model)
!wsl cmsearch RF01988_clustalw.cm RF01988_independent.fa > RF01988_clustalw_search.out
!wsl cmsearch RF01988_locarna.cm RF01988_independent.fa > RF01988_locarna_search.out

In [59]:
# Extract scores from both searches
clustalw_scores = extract_bit_scores("RF01988_clustalw_search.out")
locarna_scores = extract_bit_scores("RF01988_locarna_search.out")

print("RESULTS\n")

print(f"Clustal W CM: {len(clustalw_scores)} hits")
if clustalw_scores:
    print(f"  Mean bit score: {sum(clustalw_scores)/len(clustalw_scores):.2f}")
    print(f"  Max bit score: {max(clustalw_scores):.2f}")
    print(f"  Min bit score: {min(clustalw_scores):.2f}")
    significant_hits = [s for s in clustalw_scores if s > 50]
    print(f"  Significant hits (>50 bits): {len(significant_hits)}")

print(f"\nLocARNA CM: {len(locarna_scores)} hits")
if locarna_scores:
    print(f"  Mean bit score: {sum(locarna_scores)/len(locarna_scores):.2f}")
    print(f"  Max bit score: {max(locarna_scores):.2f}")
    print(f"  Min bit score: {min(locarna_scores):.2f}")
    significant_hits = [s for s in locarna_scores if s > 50]
    print(f"  Significant hits (>50 bits): {len(significant_hits)}")

RESULTS

Clustal W CM: 164 hits
  Mean bit score: 70.76
  Max bit score: 87.70
  Min bit score: 10.20
  Significant hits (>50 bits): 132

LocARNA CM: 142 hits
  Mean bit score: 78.95
  Max bit score: 87.60
  Min bit score: 13.40
  Significant hits (>50 bits): 132
